# Fuzzy Logic for Python 3

In my ventures into game dev, scheduling etc. I often came across the need to normalize values and apply some logic on these values. I thought that fuzzy logic would be the best tool for this job, but the libraries I found were not easily integrated into the rest of my code and not very intuitive to use.
One of my (many) approaches was to actually build fuzzy values out of native python bools or floats, but this was not supported by the language itself due to restrictions on what can be done with bool. Then I started to collect useful math functions from different places and made them usable just by pluggin them in.

The doctests in the modules should give a good idea how to use things by themselves, while here are some examples how to use everything together.

In [9]:
from fuzzy.functions import alpha, singleton, constant


a = alpha(0.2, 0.8)
s = singleton(5)



1

In [3]:
from fuzzy.classes import Domain, Set
from fuzzy.functions import alpha, triangular

numbers = Domain("numbers", low=0, high=20)
close_to_10 = Set(numbers, alpha(0.2,0.8)(triangular(0, 20)))
close_to_10.plot()

TypeError: '>=' not supported between instances of 'function' and 'float'

In [ ]:
%matplotlib inline
from fuzzy.classes import Domain, Set, Rule
from fuzzy.functions import S, R

zeit = Domain("time", low=0, high=24, res=0.1)
morgen = Set(zeit, S(0, 8))
abend = Set(zeit, R(18, 24))
mittag = ~morgen & ~abend

morgen.plot()
abend.plot()
mittag.plot()


In [ ]:
from fuzzy.classes import Domain, Set, Rule
from fuzzy.functions import S, R

dist = Domain("distance", low=0, high=1000, res=0.1)
near = Set(dist, S(0, 100))

standing = Domain("standing", low=0, high=100)
friendly = Set(standing, R(0, 100))

reaction = Domain("reaction", low=0, high=10)
aggro = IF(near & ~friendly, reaction)

## Sources
* Fuzzy Logic and Control: Software and Hardware Applications, Volume 2

      By: Mohammad Jamshidi; Nader Vadiee; Timothy J. Ross - University of New Mexico
      Publisher: Prentice Hall
      Pub. Date: June 07, 1993